In [1]:
from math import floor
import sys
import collections
name_file='ignis_zag.ino'
with open(name_file) as mfile:
    zag=mfile.read()

print('ВВедите имя будущей программы')

name_file_out=input()  ################################

file_out=open(name_file_out,'w')
file_out.write(zag)

## заполнение референсного массива цветов
cvet=[]
name_file='ignis_colors_16.txt'
with open(name_file) as mfile:
    zag1=mfile.readline()
    zag2=mfile.readline()
    for st in mfile:
        stm=[i for i in st[0:-1].split('\t')]
        stm.append(255)
        cvet.append(stm)
#print(cvet)

## чтение информация с дорожки
time=[] # время в треке
metka=[] # текст метки
person=[]
print('Введите имя дорожки отметок')
name_file=input()



# текстовая часть метки
with open(name_file) as mfile:
    for st in mfile: # проверка
        if not(len(st.split('\t'))==3):
            print('DANGER')
        else:
            #print(st.split('\t'))
            time.append(st.split('\t')[1]) 
            s=st.split('\t')[2]
            if '>' in s:
                if '/' in s:
                    chasti=s.split('/')
                    person_sbor=''
                    metka_sbor=''
                    for ch in chasti:
                        person_sbor+=ch.split('>')[0]+';'
                        metka_sbor+=ch.split('>')[1]+';'
                    person.append(person_sbor[:-1])
                    metka.append(metka_sbor[:-2])
                else:
                    person.append(s.split('>')[0])
                    metka.append(s.split('>')[1][:-1])
            else:
                person.append('all')
                metka.append(s[:-1]) 
                
## для вывода времени
stime=[]
for j in range(len(time)):
    minn=int(float(time[j])//60)
    sec=int(floor(float(time[j])%60))
    if (sec==-1):
        sec=59
        minn-=1
    msec=int(floor((float(time[j]) - floor(float(time[j])%60) - minn*60)*100))
    stime.append('{:0>2}:{:0>2}:{:0>2}'.format(minn,sec,msec))

all_persons=[]
for elem in person:
    pers=elem.split(';')
    for p in pers:
        all_persons.append(p)

persons=list(collections.Counter(all_persons).keys())
people=0
if len(persons)==1 and persons[0]=='all':
    people=1 # одна прога на всех
print()
print(metka)
print(time)
print(persons)

# конвертирование временной части в ms
dtime=[]
for j in range(len(time)-1):
    dtime.append(int( ( float(time[j+1]) - float(time[j]) )*1000))
dtime.append(120000)

# референсные аббревиатуры
comands=['colors({},{},{},{});',
        'colors_split(',
        'pulsation(',
         'dualcolors(',
         'my_cod(']
abrev=['','sp','pu','du','my']
print(comands)

# конвертирование в программу
if people==1:
    for j in range(len(metka)):
        fl=0
        if metka[j]=='end':
            file_out.write(comands[0].format(0,0,0,dtime[j]) + ' //OFF'+'\t$'+str(stime[j])+'\n')

        for a in range(1,len(abrev)):
            if abrev[a] in metka[j]:
                fl=1
                metka[j]=metka[j].replace(abrev[a],'_')
                svetas=metka[j].split('_')[0].split(',')
                schet=0
                for svet in svetas:
                    try:
                        cv=int(svet)
                        st=cvet[cv]   # теперь st это элемент цвета
                    except ValueError:
                        print('Error in metka {} in row {}'.format(metka[j],j))
                        sys.exit()
                    if schet==0:    
                        st_out=comands[a]+str(st[0])+','+str(st[1])+','+str(st[2])+','
                    else:
                        st_out+=str(st[0])+','+str(st[1])+','+str(st[2])+','
                    schet+=1

                
                hands=metka[j].split('_')  
                if not(len(hands)==1):
                    second_hand=hands[1].split(',')
                    for el in second_hand:
                        st_out+=el+','
                st_out+=str(dtime[j]) + ');' + ' //##'+str(dtime[j])+'\t$'+str(stime[j])+'\n'
                file_out.write(st_out)

        if fl==0 and not(metka[j]=='end'):
            try:
                cv=int(metka[j].split('_')[0])
                st=cvet[cv]
            except ValueError:
                print('Error in metka {} in row {}'.format(metka[j],j))
            file_out.write(comands[0].format(st[0],st[1],st[2],dtime[j])+' //##'+str(dtime[j])+'\t$'+str(stime[j])+'\n')

    file_out.write('}')
    
    
# конвертирование в программу 2 даблера
if people==0:
    # написание описания (кому какой флаг соответствует)
    num=0
    for el in persons:
        if not el=='all':
            num+=1
            file_out.write('// flag={} - {}\n'.format(num,el))
    file_out.write('int flag=0;\n')
    file_out.write('switch(flag) { \n')
    num=0
    person1=[] # массив без all
    for el in persons:
        if not el=='all':
            person1.append(el)
    persons=person1
    # написание блока для каждого человека  
    for per in persons:
        num+=1
        file_out.write('case '+str(num)+':\n')
        for j in range(len(metka)):
            flag_m=0
            if metka[j]=='end':
                file_out.write(comands[0].format(0,0,0,dtime[j])+ ' //OFF'+'\t$'+str(stime[j])+'\n')
                flag_m=1
            
            kombo_metka=metka[j].split(';')
            kombo_person=person[j].split(';')
            for m in range(len(kombo_metka)):
                fl=0
                
                for a in range(1,len(abrev)):
                    if abrev[a] in kombo_metka[m]:
                        fl=1
                        kombo_metka[m]=kombo_metka[m].replace(abrev[a],'_')
                        svetas=kombo_metka[m].split('_')[0].split(',')
                        schet=0
                        for svet in svetas:
                            try:
                                cv=int(svet)
                                st=cvet[cv]   # теперь st это элемент цветa
                            except ValueError:
                                print('Error in metka {} in row {}'.format(metka[j],j))
                                sys.exit()
                            if schet==0:    
                                st_out=comands[a]+str(st[0])+','+str(st[1])+','+str(st[2])+','
                            else:
                                st_out+=str(st[0])+','+str(st[1])+','+str(st[2])+','
                            schet+=1
                        hands=kombo_metka[m].split('_')  
                        if not(len(hands)==1):
                            second_hand=hands[1].split(',')
                            for el in second_hand:
                                st_out+=el+','
                if fl==1:
                    # добавка времени. Абсолютного и относительного.       
                    st_out+=str(dtime[j]) + ');' + ' //##'+str(dtime[j])+'\t$'+str(stime[j])+'\n'
                    # запись
                    if kombo_person[m]==per or kombo_person[m]=='all':
                        file_out.write(st_out)
                        flag_m=1
                        
                if fl==0 and not(metka[j]=='end'):
                    try:
                        cv=int(kombo_metka[m].split('_')[0])
                        st=cvet[cv]
                    except ValueError:
                        print('Error in metka {} in row {}'.format(metka[j],j))
                    if kombo_person[m]==per or kombo_person[m]=='all':
                        file_out.write(comands[0].format(st[0],st[1],st[2],dtime[j])+' //##'+str(dtime[j])+'\t$'+str(stime[j])+'\n')
                        flag_m=1
            if flag_m==0:
                file_out.write(comands[0].format(0,0,0,dtime[j])+ ' //OFF ##'+str(dtime[j])+'\t$'+str(stime[j])+'\n')
        file_out.write('break;\n\n')
    
    file_out.write("default: \n printf('Неправильный ввод переменной flag '); \n")
    file_out.write('}\n')
    file_out.write('}')


file_out.close()
print('Записано в '+name_file_out)

'''
for st in cvet:        #######################################################
    file_out.write('colors({}, {}, {}, {});\n'.format(st[0],st[1],st[2],time))
file_out.write('}')


Структура записи:   den>30 или mike>23sp45,8,10 или den>30/mike>23sp45,30 или end
'''


['0', '13', '17', '21', '34', '37;37', '6;6', '28;28', '35,21du35,40', '13,21du30,30', '40', 'end']
['3.991117', '6.482285', '10.230897', '13.980518', '17.731085', '19.625011', '21.423235', '29.072371', '34.615926', '36.481274', '43.989096', '51.478749']
['all', '1d', '2d', '3d', '4d']
['colors({},{},{},{});', 'colors_split(', 'pulsation(', 'dualcolors(', 'my_cod(']
Записано в proga_routable.ino


"\nfor st in cvet:        #######################################################\n    file_out.write('colors({}, {}, {}, {});\n'.format(st[0],st[1],st[2],time))\nfile_out.write('}')\n\n\nСтруктура записи:   den>30 или mike>23sp45,8,10 или den>30/mike>23sp45,30 или end\n"